# Putting it all Together

So far, we have had exposure to working with requests library in Python. Using our knwoledge of APIs, we are going to use the NY Times API and collect a set of data. 

First step is to [sign up](https://developer.nytimes.com/signup) for getting an API key. I have selected Top Stories for the API key request. The key code is sent to you by email. 

See a [list](https://developer.nytimes.com/) of available APIs and pick the topic you are interested in as you follow through this tutorial. 

To understand how a request can be made (GET request) NY Times has made a [GUI interaface](https://developer.nytimes.com/top_stories_v2.json) available. 

1. Fetch data from NY Times API
2. Load JSON data using requests
3. parse nested JSON for title of article and data updated
4. **Advanced** Use a visualization library of your choice, visually represent the data